In [6]:
import pandas as pd
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()
print('Iris数据集前5行：')
print(iris.data[0:5,:])
print('Iris数据集前5行分类：')
print(iris.target[0:5])

Iris数据集前5行：
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Iris数据集前5行分类：
[0 0 0 0 0]


In [7]:
#预处理阶段
from sklearn.preprocessing import MinMaxScaler
#区间缩放，返回值为缩放到[0, 1]区间的数据
iris_data=MinMaxScaler().fit_transform(iris.data)
print('Iris数据集归一化处理后前5行：')
print(iris_data[0:5,:])
iris_df=pd.DataFrame(iris_data,columns=['Sepal Length',
                                        'Sepal Width','Petal Length','Petal Width'])
iris_df['target']=iris.target

Iris数据集归一化处理后前5行：
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]]


In [8]:
#训练集与测试集分离阶段
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_df.iloc[:,0:4], 
            iris_df['target'], random_state= 14)
print('Iris训练集前5行：')
print(X_train.head(5))
print('Iris训练集分类结果：')
print(y_train.head(5).values)
print('Iris测试集前5行：')
print(X_test.head(5))
print('Iris测试集原始分类结果')
print(y_test.head(5).values)

ImportError: cannot import name 'comb'

In [ ]:
#算法实施阶段
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()    #估计器，使用KNN算法
knn.fit(X_train, y_train)
y_predicted = knn.predict(X_test)
print('Iris测试集真实结果')
print(y_test.values)
print('Iris测试集KNN算法预测结果')
print(y_predicted)

In [ ]:
#性能评估阶段
accuracy = np.mean(y_predicted == y_test) *100 
print('当前分类评估器是: knn')
print('当前Accuracy是：%.1f' %accuracy + '%')

In [ ]:
#与多种算法比较
from sklearn import tree, svm, naive_bayes,neighbors  
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, \
             RandomForestClassifier, GradientBoostingClassifier #随机森林等算法

clfs = {'svm': svm.SVC(gamma='auto'),\
        'decision_tree':tree.DecisionTreeClassifier(),
        'naive_gaussian': naive_bayes.GaussianNB(), \
        'naive_mul':naive_bayes.MultinomialNB(),\
        'K_neighbor' : neighbors.KNeighborsClassifier(),\
        'bagging_knn' : BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5,max_features=0.5), \
        'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5,max_features=0.5),
        'random_forest' : RandomForestClassifier(n_estimators=50),\
        'adaboost':AdaBoostClassifier(n_estimators=50),\
        'gradient_boost' : GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=0)
        }

#用样本数据训练模型，用测试数据测试训练后的算法正确率
def try_different_method(clf):
    clf.fit(X_train,y_train)
    score = clf.score(X_test,y_test)*100 #默认情况下分类估计器的score函数返回的是Accrracy
    print('当前Accuracy是：%.1f'%score + '%')

#依次调用clfs中的不同算法
for clf_key in clfs.keys():
    print('当前分类评估器是:',clf_key)
    clf = clfs[clf_key]
    try_different_method(clf)

In [12]:
import pandas as pd
import numpy as np

#加利福利亚房价数据
caDF = pd.read_csv('Section5 california_housing.csv')
print(caDF.shape)
print('加州房价数据前5行：')
print(caDF.head())

#观察数据，发现存在缺失值和非数值型字段
sample_incompletedata = caDF[caDF.isnull().any(axis=1)]
print('存在缺失值的数据前5行：')
print(sample_incompletedata.head())

#因为total_rooms和total_bedrooms线性相关性较强，
#为了减少运算量，将total_bedrooms列舍去
caDF.drop(['total_bedrooms'],axis=1,inplace=True)

#ocean_proximity列是文本标签，描述了房屋离海滩距离
#使用LabelEncoder将文本标签转换为数值标签
from sklearn.preprocessing import LabelEncoder
caDF[['ocean_proximity']]=caDF[['ocean_proximity']].apply\
                                          (LabelEncoder().fit_transform)

#使用SkLearn的Imputer处理缺失数据
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median") #用该列中位数填充缺失值
imputer.fit(caDF)
#将所有数据转换为float类型
caDF=pd.DataFrame(caDF,dtype=np.float)
print('整理后数据前5行，请注意比原始数据少了一列：')
print(caDF.head())

#分离训练集与测试集，median_house_value列的数据是研究的目标
from sklearn.model_selection import train_test_split
caDFdata=caDF.drop(['median_house_value'],axis=1)
caDFprice=caDF['median_house_value']
Train_X,Test_X,Train_y,Test_y=train_test_split(caDFdata,caDFprice,
                                              test_size=0.2,random_state=42)

#数据标准化,将每一列数据转换为均值为0，方差为1的数据，便于后续处理
from sklearn.preprocessing import StandardScaler
ss= StandardScaler().fit(Train_X)
Train_X = pd.DataFrame(ss.transform(Train_X), columns=Train_X.columns)
Test_X  = pd.DataFrame(ss.transform(Test_X),  columns=Test_X.columns)

print('标准化后训练集前5行：')
print(Train_X.head())

(20640, 10)
加州房价数据前5行：
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  
存在缺失值的数据前5行：
     longitude  latitude  housing_media

ImportError: cannot import name '_object_dtype_isnan'

In [ ]:
#随机森林回归，是集成了多棵决策树而成的森林
from sklearn import ensemble
model_rf= ensemble.RandomForestRegressor(n_estimators=20)  # 使用20个决策树
model_rf.fit(Train_X,Train_y)
rf_score=model_rf.score(Test_X,Test_y)*100
print('sklearn随机森林模型得分: %.1f' %rf_score + '%')
rf_pred=model_rf.predict(Test_X)

In [ ]:
#人工神经网络回归，Sklearn中，又称多层感知机MLP
from sklearn.neural_network import MLPRegressor  
model_mlp = MLPRegressor(solver='lbfgs', hidden_layer_sizes=(5, 5),
                         random_state=1)
model_mlp.fit(Train_X,Train_y)
mlp_score=model_mlp.score(Test_X,Test_y)*100
print('sklearn人工神经网络回归模型得分: %.1f' %mlp_score + '%')

In [ ]:
#采用GridSearchCV来进行参数调整实验，找出最佳参数组合
from sklearn.model_selection import GridSearchCV
param_grid = {'solver':['lbfgs','sgd','adam'],
              'hidden_layer_sizes': [(5,5),(10,10)]
             }
#对param_grid中的各参数进行组合，传递进MPL回归器。
#cv=3,3折交叉验证，将数据集随机分为3份，每次将一份作为测试集，其他为训练集
#n_jobs=-1，使用CPU核心数，-1表示所有可用的核
best_mlp =GridSearchCV(MLPRegressor(max_iter=200),param_grid,cv=3,n_jobs=-1)
best_mlp.fit(Train_X,Train_y)
print('当前最佳参数组合：',best_mlp.best_params_)
best_score=best_mlp.score(Test_X,Test_y)*100
print('sklearn人工神经网络上述参数得分: %.1f' %best_score + '%')
#用以上模型对Test_X进行预测
mlp_pred = best_mlp.predict(Test_X)

In [ ]:
#绘图比较模型预测房价与真实房价的差异
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False 
fig = plt.figure(figsize=(10, 6))  # dpi参数指定绘图对象的分辨率，即每英寸多少个像素，缺省值为80
#axes = fig.add_subplot(1, 1, 1)
#为便于观察，只取部分结果作图
T1=Test_y[:50]
P1=mlp_pred[:50]
R1=rf_pred[:50]
plt.plot(range(len(T1)),T1, 'g',label='实际',linewidth=3)
plt.plot(range(len(P1)),P1, 'y--',label='神经网络回归',linewidth=2)
plt.plot(range(len(P1)),R1, 'r:',label='随机森林回归',linewidth=2)
fig.tight_layout()
plt.legend()
plt.title('sklearn 回归模型')
plt.savefig('skl_01.png',dpi=300,bbox_inches='tight')
plt.show()